In [1]:
from pyspark import SparkConf ,SparkContext
from pyspark.sql.session import SparkSession


conf = SparkConf().setMaster("local").setAppName("word-counts")

sc = SparkContext(conf=conf)

In [2]:
import os
import re
d = {}
documentsProcessedRDD = sc.emptyRDD()
for i in range(len(os.listdir("dataset"))):
    bookRawRDD = sc.textFile("dataset/" + str(i))
    bookMessyTokensRDD = bookRawRDD.flatMap(lambda line: line.lower().strip().split(" "))
    bookCleanTokensRDD = bookMessyTokensRDD.map(lambda letter: re.sub(r'[^A-Za-z]', '', letter))
    
    #remove stop words
    from pyspark.ml.feature import StopWordsRemover
    remover = StopWordsRemover()
    stopwords = remover.getStopWords()
    bookwordsRDD = bookCleanTokensRDD.filter(lambda PointLessW: PointLessW not in stopwords)
    ##Remove the empty places
    bookEmptyRemoveRDD = bookwordsRDD.filter(lambda x: x != "")
    
    ### Appending the Individual book RDD to documentsProcessedRDD
    documentsProcessedRDD = documentsProcessedRDD.union(bookEmptyRemoveRDD)
    #print(documentsProcessedRDD.count())

In [4]:
print((documentsProcessedRDD.count()))

2741322


In [3]:
# map words to immediate keyvlue pairs
documentsProcessedPairsRDD = documentsProcessedRDD.map(lambda word: (word,1))

In [4]:
# tranform pairs to word count
documentsProcessedWordCountRDD = documentsProcessedPairsRDD.reduceByKey(lambda acc, value: acc + value)

In [ ]:
# sort words in decending order
documentsProcessedResults = documentsProcessedWordCountRDD.map(lambda x: (x[1], x[0])).collect()
#print(documentsProcessedResults)

import pandas as pd
df = pd.DataFrame(documentsProcessedResults)
#df = df.insert(0, "ID", range(0, len(df)))
#print(df)
df = df.assign(New_ID=[ i for i in range(len(df))])[['New_ID'] + df.columns.tolist()]
del df['0']
df

In [28]:
print(type(documentsProcessedResults[0]))

<class 'tuple'>


In [33]:
rdd = sc.parallelize([1, 1, 2, 3])

r =rdd.map(lambda x, value: x+value).collect()
print(r)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 114.0 failed 1 times, most recent failure: Lost task 0.0 in stage 114.0 (TID 2485, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/vignesh/spark-2.4.7-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/vignesh/spark-2.4.7-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/vignesh/spark-2.4.7-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/vignesh/spark-2.4.7-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
TypeError: